# Workflow 1, Module 2

One approach to solving this module is to not define quite so tightly what's going on at the subquestion level in terms of enrichements and archetypes and so on, but simply pass the question to ROBOKOP and let its scoring bring the the best answers to the top.  Here we will use the quick service to start with a disease, find relevant phenotypes, and from there find genetic conditions. The answers will come out ranked by path.

For more details, see the "quick" notebook in greengamma/general.

First, we'll have a quick function that calls the quick service, and some functions for properly creating the question. 

In [2]:
import requests
import pandas as pd

def quick(question):
    url=f'http://robokop.renci.org:80/api/simple/quick/'
    response = requests.post(url,json=question)
    print( f"Return Status: {response.status_code}" )
    if response.status_code == 200:
        return response.json()
    return response

In [3]:
def make_N_step_question(types,curies):
    question = {
                'machine_question': {
                    'nodes': [],
                    'edges': []
                }
            }
    for i,t in enumerate(types):
        newnode = {'id': i, 'type': t}
        if curies[i] is not None:
            newnode['curie'] = curies[i]
        question['machine_question']['nodes'].append(newnode)
        if i > 0:
            question['machine_question']['edges'].append( {'source_id': i-1, 'target_id': i})
    return question

In [4]:
def extract_final_nodes(returnanswer):
    nodes = [{'node_name': answer['nodes'][2]['name'], 'node_id': answer['nodes'][2]['id']} for answer in returnanswer['answers']]
    return pd.DataFrame(nodes)

## Without pathway expansion

The module as written starts with a genetic condition, gets genes, and then gets chemicals interacting with those genes.   On the presumption that this is insufficient to get into new chemical space, it also adds the expansion from genes to other genes via pathways.   But lets originally start without the pathway expansion.  So we'll just do 

`genetic_condition` to `gene` to `chemical_substance`

In [5]:
glucose_intolerance = 'MONDO:0001076' #glucose intolerance
cf = 'MONDO:0009061' #cystic fibrosis

In [6]:
gi_question = make_N_step_question(['genetic_condition','gene','chemical_substance'],[glucose_intolerance,None,None])
gi_answer = quick(gi_question)

Return Status: 200


In [7]:
gi_frame = extract_final_nodes(gi_answer)
gi_frame

,node_id,node_name
0,CHEBI:6801,Metformin
1,CHEBI:6801,Metformin
2,CHEBI:17154,Niacinamide
3,CHEBI:18406,AICA ribonucleotide
4,CHEBI:18406,AICA ribonucleotide
5,CHEBI:5001,Fenofibrate
6,CHEBI:94686,orlistat
7,CHEBI:41879,Dexamethasone
8,CHEBI:6801,Metformin
9,CHEBI:29678,sodium arsenite


## With pathway expansion

We can also run the question as asked by opening up the genes to those involved in the same biological processes

In [8]:
gi_exp_question = make_N_step_question(['genetic_condition','gene','biological_process_or_activity','gene','chemical_substance'],[glucose_intolerance,None,None,None,None])
gi_exp_answer = quick(gi_exp_question)

Return Status: 500


In [9]:
gi_exp_frame = extract_final_nodes(gi_exp_answer)
gi_exp_frame

TypeError: 'Response' object is not subscriptable